In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from sklearn.metrics import confusion_matrix

In [2]:
print("GPU Available: ", tf.test.is_gpu_available())
print(tf.__version__)
# tf.debugging.set_log_device_placement(True)

GPU Available:  True
2.0.0


In [3]:
# warnings.filterwarnings('ignore')
tf.keras.backend.clear_session()  # For easy reset of notebook state.
np.set_printoptions(suppress=True, linewidth=120, precision=2)

In [4]:
def perf_measure(y_true, y_pred):
    
    cnf_matrix = confusion_matrix(y_true, y_pred)
    
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    
    FSCORE = np.divide((2*PPV*TPR), (PPV+TPR))
    
    return PPV, TPR, FSCORE, FNR, FPR, TNR

In [5]:
name_of_particle = 'Egammas'

X_train = np.load("matrices/" + name_of_particle +"_train.npy",).astype('float32')
y_train = np.load("matrices/" + name_of_particle +"_y_train.npy",).astype('float32')
X_val = np.load("matrices/" + name_of_particle +"_val.npy",).astype('float32')
y_val = np.load("matrices/" + name_of_particle +"_y_val.npy",).astype('float32')
X_test = np.load("matrices/" + name_of_particle +"_test.npy",).astype('float32')
y_test = np.load("matrices/" + name_of_particle +"_y_test.npy",).astype('float32')
X_train = X_train[:, :-3]
X_val = X_val[:, :-3]
X_test = X_test[:, :-3]
N, V = X_train.shape
K = 1
V

211

In [6]:
supp_min = np.min(X_train, keepdims=True, axis=0)
supp_max = np.max(X_train, keepdims=True, axis=0)

In [7]:
batch_size = 512
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)  #.shuffle(1000)  

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size) #.shuffle(1000)  #shuffle(1000)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)  #.shuffle(1000)


In [8]:
class OccModel(Model):
    
    def __init__(self):
        
        super(OccModel, self).__init__()
        
        self.dense1 = tf.keras.layers.Dense(256, input_shape=(V,),
                                            kernel_initializer='uniform', name='dense1')
        
        self.dense2 = tf.keras.layers.Dense(32,
                                           kernel_initializer='uniform',
                                            name='dense2')
        
        self.pred_layer = tf.keras.layers.Dense(1, activation='linear',
                                           kernel_initializer='uniform', 
                                                name='predictions')   
        
    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return self.pred_layer(x)
    

model_occ = OccModel()

### Brute-forth

#### Creating the custom loss function for training: 

In [9]:
# (*)
def loss_bf(model, X_tr, y_tr, x_pos, x_neg): 
    
#     x_pos = X_tr[np.where(y_tr==0)]
#     x_neg = X_tr[np.where(y_tr==1)]
    
    p_pos = model(x_pos)
    p_neg = model(x_neg)
    
    N_pos, V = x_pos.shape
    N_neg, _ = x_neg.shape
    
#     supp_min = tf.reduce_min(x_neg, keepdims=True, axis=0)
#     supp_max = tf.reduce_max(x_neg, keepdims=True, axis=0)
    
    x_pseudo = tf.random.uniform(shape=[N_pos, V],
                                 minval=supp_min-3, maxval=supp_max+3,
                                 dtype='float32',)
    
    p_psedo = model_occ(x_pseudo)
    
    loss_pos = N_pos / (N_pos + N_neg) * tf.reduce_mean(tf.nn.softplus(-p_pos))
    loss_neg = N_neg / (N_pos + N_neg) * tf.reduce_mean(tf.nn.softplus(p_neg)) 
    loss_pseudo = 0.001 * tf.reduce_mean(tf.nn.softplus(p_psedo))
    
    preds = tf.nn.sigmoid(model_occ(X_tr))
    
    return loss_pos + loss_neg + loss_pseudo, preds

In [10]:
# def loss_bf(model, X_tr, y_tr,):  
    
#     x_pos = X_train[np.where(y_train==0)]
#     x_neg = X_train[np.where(y_train==1)]
    
#     p_pos = model(x_pos)
#     p_neg = model(x_neg)
    
#     N_pos, V = x_pos.shape
#     N_neg, _ = x_neg.shape
    
#     supp_min = tf.reduce_min(x_neg, keepdims=True, axis=0)
#     supp_max = tf.reduce_max(x_neg, keepdims=True, axis=0)
    
#     x_pseudo = tf.random.uniform(shape=[N_pos, V],
#                                  minval=supp_min-3, maxval=supp_max+3,
#                                  dtype='float32',)
    
#     p_psedo = model_occ(x_pseudo)
    
#     loss_pos = N_pos / (N_pos + N_neg) * (tf.nn.softplus(-p_pos))
#     loss_neg = N_neg / (N_pos + N_neg) * (tf.nn.softplus(p_neg))
#     loss_pseudo = 0.01 * (tf.nn.softplus(p_psedo))
    
#     print("loss_bf pos:", loss_pos,)
#     print(" ")
#     print("loss_bf neg:", loss_neg,)
#     print(" ")
#     print("loss_bf pseudo:", loss_pseudo)
#     print(" ")
    
#     preds = tf.nn.sigmoid(model_occ(X_tr))
#     return loss_pos, loss_neg, loss_pseudo, preds

#### Chosing an optimizer 

In [11]:
loss_object = tf.keras.losses.BinaryCrossentropy() 
optimizer = tf.keras.optimizers.Adam(1e-6) # 2e-4

#### metrics to measure the loss and accuracy of the model

In [12]:
train_loss = tf.keras.metrics.Mean(name='train_loss') 
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')

In [13]:
# (*)
@tf.function
def train_step(X_tr, y_tr, x_pos, x_neg): 
    with tf.GradientTape() as tape:  # Record operations for automatic differentiation
        loss, preds = loss_bf(model=model_occ, X_tr=X_tr, y_tr=y_tr, x_pos=x_pos, x_neg=x_neg)  # x_pos=x_pos, x_neg=x_neg, x_pseudo=x_pseudo
        # y_tr = tf.reshape(tf.tile(y_tr, [2]), [-1, 2])
        loss_ = loss_object(y_tr, preds)
        
    gradients = tape.gradient(loss, model_occ.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model_occ.trainable_variables))
    
    train_loss(loss_)
    train_accuracy(y_tr, preds)
    

In [14]:
# @tf.function
# def train_step(X_tr, y_tr):  # x_pos, x_neg, x_pseudo
#     with tf.GradientTape() as tape_pos, tf.GradientTape() as tape_neg, tf.GradientTape() as tape_psuedo:  # Record operations for automatic differentiation
#         loss_pos, loss_neg, loss_pseudo, preds = loss_bf(model=model_occ, X_tr=X_tr, y_tr=y_tr )  
#         labels = y_tr # tf.reshape(tf.tile(y_tr, [2]), [-1, 2])
#         loss_ = loss_object(labels, preds)
        
#     gradients_pos = tape_pos.gradient(loss_pos, model_occ.trainable_variables)
#     gradients_neg = tape_neg.gradient(loss_neg, model_occ.trainable_variables)
#     gradients_psuedo = tape_psuedo.gradient(loss_pseudo, model_occ.trainable_variables)
# #     gradients = gradients_pos + gradients_neg + gradients_psuedo
# #     print("gradients:", gradients)
# #     optimizer.apply_gradients(zip(gradients, model_occ.trainable_variables))

#     print("gradients_pos:", gradients_pos)
#     print(" ")
#     print("gradients_neg:", gradients_neg)
#     print(" ")
#     print("gradients_psuedo:", gradients_psuedo)
#     print(" ")


#     optimizer.apply_gradients(zip(gradients_pos, model_occ.trainable_variables))
#     optimizer.apply_gradients(zip(gradients_neg, model_occ.trainable_variables))
#     optimizer.apply_gradients(zip(gradients_psuedo, model_occ.trainable_variables))
    
    
#     train_loss(loss_)
#     train_accuracy(labels, preds)
    

In [15]:
@tf.function
def test_step(X, labels):
    predictions = model_occ(X)
#     labels = tf.reshape(tf.tile(labels, [2]), [-1, 2])
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [17]:
EPOCHS = 200

training_losses_occ, training_accuracies_occ = [], []
validations_losses_occ, validations_accuracies_occ = [], []

for epoch in range(EPOCHS):
    for X_tr, y_tr in train_ds:
        x_pos = tf.convert_to_tensor(X_tr.numpy()[np.where(y_tr.numpy()==0)])
        x_neg = tf.convert_to_tensor(X_tr.numpy()[np.where(y_tr.numpy()==1)])
        results = train_step(X_tr=X_tr, y_tr=y_tr, x_pos=x_pos, x_neg=x_neg)
        
        training_losses_occ.append(train_loss.result().numpy())
        training_accuracies_occ.append(train_accuracy.result().numpy())
        
    for X_val, y_val in val_ds:
        test_step(X_val, y_val)
        
        validations_losses_occ.append(test_loss.result().numpy())
        validations_accuracies_occ.append(test_accuracy.result().numpy())
    
    template = 'Epoch {}, Train Loss: {:.3f}, Train Accuracy:{:.3f}, \
    Validation Loss: {:.3f}, Validation Accuracy:{:.3f},'
    
    print (template.format(epoch+1,
                         train_loss.result().numpy(),
                         train_accuracy.result().numpy()*100,
                           
                         test_loss.result().numpy(),
                         test_accuracy.result().numpy()*100),)
    
     # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    
model_occ.save_weights("NN-ckecks/OCC_Bope"+ name_of_particle +".h5")

In [17]:
model_occ.summary()

Model: "occ_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense1 (Dense)               multiple                  54272     
_________________________________________________________________
dense2 (Dense)               multiple                  8224      
_________________________________________________________________
predictions (Dense)          multiple                  33        
Total params: 62,529
Trainable params: 62,529
Non-trainable params: 0
_________________________________________________________________


In [18]:
new_model_occ = OccModel()
new_model_occ.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6))

# Since In this implementation instead of weight we are dealing 
# with codes and classes therefore the traditional serialization and
# deserialization is not possible. So we have to first initialze
# the model (which is code) and then load the weights 
# Ref: https://colab.research.google.com/drive/172D4jishSgE3N7AO6U2OKAA_0wNnrMOq#scrollTo=OOSGiSkHTERy

cntr = 0
for i, j in train_ds:
    if cntr == 0:
        new_model_occ.train_on_batch(i[:1], j[:1])
    cntr += 1 

# new_model_occ.load_weights('NN-ckecks/ThreeLayerNN_model'+ name_of_particle+'.h5')
test_predictions = new_model_occ.predict(X_test)
probabilities = tf.nn.sigmoid(test_predictions)
labels_pred_occ = tf.argmax(probabilities, axis=1)


labels_true_occ = []
for i, j in test_ds:
    for k in j.numpy():
        labels_true_occ.append(k)

PPV3, TPR3, FSCORE3, FNR3, FPR3, TNR3 = perf_measure(y_true=labels_true_occ, y_pred=labels_pred_occ)


PPV3, TPR3, FSCORE3, FNR3, TNR3, # FPR3, 

/usr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide
/usr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide


(array([0.99,  nan]),
 array([1., 0.]),
 array([ 1., nan]),
 array([0., 1.]),
 array([0., 1.]))

In [19]:
from sklearn.metrics import roc_curve

In [20]:
roc_curve(labels_true_occ, labels_pred_occ)

(array([0., 1.]), array([0., 1.]), array([1, 0]))

In [21]:
N= np.array([1, 0, 3, 0])
D = np.array([1, 2, 0, 0])

In [22]:
N/D

/usr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
/usr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([ 1.,  0., inf, nan])

In [23]:
# class sanity_check(object):
#     def __init__(self):
#         super(sanity_check, self).__init__
#         self.W = tf.Variable(tf.random.normal(shape=[2, 3]), name='W')
#         self.b = tf.Variable(tf.random.normal(shape=[2,]), name='b')
    
#     def __call__(self, x):
#         return self.W*x
        
# model = sanity_check()

In [24]:
# W = tf.Variable(tf.random.normal(shape=[2, 3]), name='W')
# W

In [25]:
# x= tf.Variable(np.array([[1, 2, 3], [10, 12, 13],]).astype("float32"))
# x

In [26]:
# model = sanity_check()


In [27]:
# model(x)